<a href="https://colab.research.google.com/github/Stefano-Previti/Curiosity_driven_exploration_by_self_supervised_prediction/blob/main/Curiosity_driven_exploration_by_Self_supervised_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**▶INITIAL SETUP OF THE ENVIROMENT**

In [ ]:
# Super Mario environment for OpenAI Gym
!pip install gym-super-mario-bros

# Backend for NES emulation
!pip install nes-py

# PyTorch for building and training neural networks
!pip install torch torchvision

# TensorBoard for monitoring training progress
!pip install tensorboard

# Visualization tools for plotting and analysis
!pip install matplotlib


In [ ]:
!pip install pyvirtualdisplay


In [16]:
from pyvirtualdisplay import Display
import gym
from gym_super_mario_bros import make
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
from gym.wrappers import FrameStack
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
import cv2
import random
import time
from gym.spaces import Box
# Wrapper to skip frames
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum rewards, and return the last observation"""
        total_reward = 0.0
        done = False
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info


# Wrapper to convert frames to grayscale
class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        return cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)


# Wrapper to resize frames
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        self.shape = (shape, shape) if isinstance(shape, int) else tuple(shape)
        obs_shape = self.shape + (1,)  # Add channel dimension for grayscale
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        observation = cv2.resize(observation, self.shape, interpolation=cv2.INTER_AREA)
        return np.expand_dims(observation, axis=-1)  # Add channel dimension


# Updated Super Mario Environment Class
class SuperMarioEnvironment:
    def __init__(self, level_id="SuperMarioBros-1-1-v0", frame_size=(42, 42), skip_frames=4):
        """Initialize the Super Mario Environment."""
        self.level_id = level_id
        self.frame_size = frame_size
        self.skip_frames = skip_frames
        self.env = None
        self.actions = COMPLEX_MOVEMENT

    def create_environment(self):
        """Create the Super Mario environment with frame skipping, grayscale, and resizing."""
        self.env = make(self.level_id)
        self.env = SkipFrame(self.env, skip=self.skip_frames)  # Skip frames
        self.env = GrayScaleObservation(self.env)  # Convert to grayscale
        self.env = ResizeObservation(self.env, shape=self.frame_size)  # Resize frames
        self.env = FrameStack(self.env, num_stack=4)  # Stack frames
        print(f"Environment for {self.level_id} created.")

    def reset_environment(self):
        """Reset the environment and return the initial state."""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        return self.env.reset()

    def step(self, action):
        """Take an action in the environment."""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        return self.env.step(action)

    def render_frame(self):
        """Render the current frame in RGB format."""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        return self.env.render(mode='rgb_array')

    def close(self):
        """Close the environment."""
        if self.env:
            self.env.close()
    @property
    def action_space(self):
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment() first.")
        return self.env.action_space

In [18]:
# Create the game environment
env = SuperMarioEnvironment(frame_size=(42, 42), skip_frames=4)
env.create_environment()

# Get frame dimensions
state = env.reset_environment()
frame = env.render_frame()
frame_height, frame_width, _ = frame.shape

# Configure video writer
video_filename = "super_mario.mp4"
fps = 20  # Frames per second
video_writer = cv2.VideoWriter(
    video_filename,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (frame_width, frame_height),
)

# Frame buffer
frame_buffer = []
frame_time = 1 / fps  # Time per frame

try:
    start_time = time.time()
    for step in range(1000):  # Number of steps to simulate
        # Perform a random action from the action space
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)

        # Capture frame and add to buffer
        frame = env.render_frame()  # Get the current rendered frame
        frame_buffer.append(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

        # Handle resets
        if done:
            print(f"Game Over at step {step}. Restarting...")
            state = env.reset_environment()
            continue  # Skip writing frames during reset

        # Synchronize frame timing
        elapsed_time = time.time() - start_time
        if elapsed_time < frame_time:
            time.sleep(frame_time - elapsed_time)
        start_time = time.time()

    # Write all buffered frames to the video file
    for frame in frame_buffer:
        video_writer.write(frame)

finally:
    env.close()
    video_writer.release()
    print(f"Video saved as {video_filename}")



Environment for SuperMarioBros-1-1-v0 created with optimized wrappers.
Game Over at step 208. Restarting...
Video saved as super_mario.mp4


**▶ICM**